Useful links

1. For the architecture https://towardsdatascience.com/deep-learning-for-specific-information-extraction-from-unstructured-texts-12c5b9dceada
2. https://androidkt.com/multi-label-text-classification-in-tensorflow-keras/
3. https://keras.io/preprocessing/sequence/
4. https://machinelearningmastery.com/develop-word-based-neural-language-models-python-keras/ ( Not really)
5. For deep learning using word embeddings https://stackabuse.com/python-for-nlp-multi-label-text-classification-with-keras/



In [1]:
import spacy
import pandas as pd
from tqdm import tqdm

In [2]:
DATA_DIR = "../../data/processed/"
INPUT_FILE_NAME = 'final_squash15_with_pos_ner_tm.parquet'

In [3]:
df = pd.read_parquet(DATA_DIR + INPUT_FILE_NAME)
df.head()

,speaker,headline,description,duration,tags,transcript,WC,clean_transcript,clean_transcript_string,sim_tags,squash15_tags,pos_sequence,ner_sequence,tm
0,Al Gore,Averting the climate crisis,With the same humor and humanity he exuded in ...,0:16:17,"cars,alternative energy,culture,politics,scien...","0:14\r\r\rThank you so much, Chris.\rAnd it's ...",2281.0,"b'[""thank"", ""chris"", ""truly"", ""great"", ""honor""...",thank chris truly great honor opportunity come...,"cars,solar system,energy,culture,politics,scie...","culture,politics,science,global issues,technology",VERB PROPN ADV ADJ NOUN NOUN VERB NOUN ADV ADV...,PERSON ORG ORG GPE LOC ORG PRODUCT GPE GPE PER...,"[0.04325945698517057, 0.0, 0.00142482934694180..."
1,Amy Smith,Simple designs to save a life,Fumes from indoor cooking fires kill more than...,0:15:06,"MacArthur grant,simplicity,industrial design,a...","0:11\r\r\rIn terms of invention,\rI'd like to ...",2687.0,"b'[""term"", ""invention"", ""like"", ""tell"", ""tale""...",term invention like tell tale favorite project...,"macarthur grant,simplicity,design,solar system...","design,global issues",NOUN NOUN SCONJ VERB PROPN ADJ NOUN VERB NOUN ...,GPE DATE CARDINAL DATE ORG PERSON LOC ORG GPE ...,"[0.013287880838036227, 0.0, 0.0, 0.00511725094..."
2,Ashraf Ghani,How to rebuild a broken state,Ashraf Ghani's passionate and powerful 10-minu...,0:18:45,"corruption,poverty,economics,investment,milita...","0:12\r\r\rA public, Dewey long ago observed,\r...",2506.0,"b'[""public"", ""dewey"", ""long"", ""ago"", ""observe""...",public dewey long ago observe constitute discu...,"corruption,inequality,science,investment,war,c...","science,culture,politics,global issues,business",ADJ PROPN ADV ADV VERB ADJ NOUN NOUN PROPN PRO...,DATE NORP ORDINAL DATE MONEY DATE DATE DATE EV...,"[0.0, 0.006699599134802422, 0.0, 0.00564851883..."
3,Burt Rutan,The real future of space exploration,"In this passionate talk, legendary spacecraft ...",0:19:37,"aircraft,flight,industrial design,NASA,rocket ...","0:11\r\r\rI want to start off by saying, Houst...",3092.0,"b'[""want"", ""start"", ""say"", ""houston"", ""problem...",want start say houston problem enter second ge...,"flight,design,nasa,science,invention,entrepren...","design,science,business",VERB NOUN VERB PROPN NOUN VERB ADJ NOUN NOUN N...,GPE ORDINAL ORG PERSON DATE DATE DATE TIME PER...,"[0.040282108339079505, 0.03732895646484358, 0...."
4,Chris Bangle,Great cars are great art,American designer Chris Bangle explains his ph...,0:20:04,"cars,industrial design,transportation,inventio...","0:12\r\r\rWhat I want to talk about is, as bac...",3781.0,"b'[""want"", ""talk"", ""background"", ""idea"", ""car""...",want talk background idea car art actually mea...,"cars,design,transportation,invention,technolog...","design,technology,business,science",VERB NOUN NOUN NOUN NOUN NOUN ADV ADJ NOUN NOU...,PERSON PRODUCT ORG ORG PERSON PERSON PERSON OR...,"[0.08049208168957463, 0.0, 0.0, 0.008031187136..."


In [4]:
df.iloc[:,:14].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2313 entries, 0 to 2312
Data columns (total 14 columns):
speaker                    2313 non-null object
headline                   2313 non-null object
description                2313 non-null object
duration                   2313 non-null object
tags                       2313 non-null object
transcript                 2313 non-null object
WC                         2313 non-null float64
clean_transcript           2313 non-null object
clean_transcript_string    2313 non-null object
sim_tags                   2313 non-null object
squash15_tags              2313 non-null object
pos_sequence               2313 non-null object
ner_sequence               2313 non-null object
tm                         2313 non-null object
dtypes: float64(1), object(13)
memory usage: 253.1+ KB


In [5]:
def print_full_dataframe(x):
    pd.set_option('display.max_rows', len(x))
    print(x)
    pd.reset_option('display.max_rows')
    
def compute_tag_ratio(target_column, df=df):
    tags = df[target_column].str.replace(', ',',').str.lower().str.strip()
    split_tags = tags.str.split(',')
    tag_counts_per_talk = split_tags.apply(len)

    joined_tags = tags.str.cat(sep=',').split(',')
    all_tags = pd.Series(joined_tags)

    tag_counts = all_tags.value_counts().rename_axis(target_column).reset_index(name='counts')
    tag_counts['no_count'] = len(df)-tag_counts['counts']
    tag_counts['ratio'] = tag_counts['counts']/tag_counts['no_count']
    tag_counts['overall_ratio'] = tag_counts['counts']/(tag_counts['no_count'] + tag_counts['counts'])
    return tag_counts

#print(compute_tag_ratio('squash3_tags', df))
squashed_tag_counts = compute_tag_ratio('squash15_tags', df)
print_full_dataframe(squashed_tag_counts)

    squash15_tags  counts  no_count     ratio  overall_ratio
0         science    1467       846  1.734043       0.634241
1         culture    1155      1158  0.997409       0.499351
2      technology     787      1526  0.515727       0.340251
3   global issues     679      1634  0.415545       0.293558
4          design     477      1836  0.259804       0.206226
5         history     385      1928  0.199689       0.166450
6        business     349      1964  0.177699       0.150886
7   entertainment     285      2028  0.140533       0.123217
8           media     279      2034  0.137168       0.120623
9    biomechanics     220      2093  0.105112       0.095115
10         future     218      2095  0.104057       0.094250
11   biodiversity     218      2095  0.104057       0.094250
12       humanity     217      2096  0.103531       0.093818
13       politics     199      2114  0.094134       0.086035
14  communication     185      2128  0.086936       0.079983


# 3. Feature Extraction via Deep learning

## 3.1 Create one hot encoding

In [6]:
# from sklearn.preprocessing import MultiLabelBinarizer

# y = []
# for index, row in df.iterrows():
#     y.append(set(row['squash3_tags'].split(',')))
    
# mlb = MultiLabelBinarizer()
# encoded_y = mlb.fit_transform(y)

In [7]:
# print(encoded_y[0])
# print(len(encoded_y[0]))

In [8]:
joined_tags = df['squash15_tags'].str.cat(sep=',').split(',')
all_tags = pd.Series(joined_tags).str.strip().str.lower()
all_tags = list(dict.fromkeys(all_tags))
try:
    all_tags.remove('')
except:
    pass
print(all_tags)
print(len(all_tags))

['culture', 'politics', 'science', 'global issues', 'technology', 'design', 'business', 'biomechanics', 'biodiversity', 'media', 'entertainment', 'history', 'future', 'communication', 'humanity']
15


In [9]:
def create_one_hot_encode(df=df):
    complete_transcripts_tags = []
    for rows, value in df.iterrows():
        one_hot_encoding = [0] * len(all_tags)
        headline = [value['headline']]
        transcript = [value['clean_transcript_string']]
        pos_sequence = [value['pos_sequence']]
        ner_sequence = [value['ner_sequence']]
        tm = [value['tm']]
        indiv_tags = value['squash15_tags'].split(',')
        for tags in indiv_tags:
            if tags == '':
                continue
            index = all_tags.index(tags.lower().lstrip(' '))
            one_hot_encoding[index] = 1
        indiv_transcript_tags = headline + transcript + pos_sequence + ner_sequence + tm +one_hot_encoding
        complete_transcripts_tags.append(indiv_transcript_tags)
    return pd.DataFrame(complete_transcripts_tags, columns=['headline', 'transcript', 'pos_sequence', 'ner_sequence','tm'] + all_tags)

In [10]:
df = create_one_hot_encode()
df

,headline,transcript,pos_sequence,ner_sequence,tm,culture,politics,science,global issues,technology,design,business,biomechanics,biodiversity,media,entertainment,history,future,communication,humanity
0,Averting the climate crisis,thank chris truly great honor opportunity come...,VERB PROPN ADV ADJ NOUN NOUN VERB NOUN ADV ADV...,PERSON ORG ORG GPE LOC ORG PRODUCT GPE GPE PER...,"[0.04325945698517057, 0.0, 0.00142482934694180...",1,1,1,1,1,0,0,0,0,0,0,0,0,0,0
1,Simple designs to save a life,term invention like tell tale favorite project...,NOUN NOUN SCONJ VERB PROPN ADJ NOUN VERB NOUN ...,GPE DATE CARDINAL DATE ORG PERSON LOC ORG GPE ...,"[0.013287880838036227, 0.0, 0.0, 0.00511725094...",0,0,0,1,0,1,0,0,0,0,0,0,0,0,0
2,How to rebuild a broken state,public dewey long ago observe constitute discu...,ADJ PROPN ADV ADV VERB ADJ NOUN NOUN PROPN PRO...,DATE NORP ORDINAL DATE MONEY DATE DATE DATE EV...,"[0.0, 0.006699599134802422, 0.0, 0.00564851883...",1,1,1,1,0,0,1,0,0,0,0,0,0,0,0
3,The real future of space exploration,want start say houston problem enter second ge...,VERB NOUN VERB PROPN NOUN VERB ADJ NOUN NOUN N...,GPE ORDINAL ORG PERSON DATE DATE DATE TIME PER...,"[0.040282108339079505, 0.03732895646484358, 0....",0,0,1,0,0,1,1,0,0,0,0,0,0,0,0
4,Great cars are great art,want talk background idea car art actually mea...,VERB NOUN NOUN NOUN NOUN NOUN ADV ADJ NOUN NOU...,PERSON PRODUCT ORG ORG PERSON PERSON PERSON OR...,"[0.08049208168957463, 0.0, 0.0, 0.008031187136...",0,0,1,0,1,1,1,0,0,0,0,0,0,0,0
5,Sampling the ocean's DNA,break ask people comment age debate comment un...,VERB VERB NOUN VERB NOUN NOUN NOUN VERB NOUN A...,DATE DATE ORG DATE DATE PERSON ORG CARDINAL CA...,"[0.0, 0.01122282724927712, 0.0, 0.163765591818...",0,0,1,0,1,0,0,1,1,0,0,0,0,0,0
6,Simplicity sells,music sound silence simon garfunkel hello voic...,NOUN PROPN PROPN PROPN PROPN INTJ NOUN NOUN AD...,PERSON TIME TIME ORG PERSON FAC DATE DATE ORG ...,"[0.062272408748748564, 0.0, 0.0243049615007748...",0,0,1,0,1,1,0,0,0,1,1,0,0,0,0
7,A memorial at Ground Zero,kurt andersen like architect david hog limelig...,PROPN PROPN SCONJ PROPN PROPN PROPN PROPN ADV ...,PERSON PERSON ORG PERSON DATE GPE PERSON PERSO...,"[0.045631610155157765, 0.0, 0.0, 0.0, 0.004847...",1,0,0,0,0,1,0,0,0,0,0,0,0,0,0
8,To invent is to give,point time come learn morning world expert gue...,NOUN NOUN VERB VERB NOUN NOUN NOUN VERB ADJ NO...,DATE DATE TIME PERSON DATE ORG CARDINAL GPE CA...,"[0.041662618917564086, 0.0, 0.0, 0.0, 0.004215...",1,0,1,1,1,1,1,0,0,1,0,0,0,0,0
9,The killer American diet that's sweeping the p...,legitimate concern aid avian flu hear brillian...,ADJ NOUN NOUN ADJ NOUN VERB ADJ PROPN ADJ ADV ...,NORP TIME LOC LOC CARDINAL DATE DATE DATE NORP...,"[0.003366184031329983, 0.0, 0.0007976442417315...",1,0,1,1,0,0,0,0,0,0,0,0,0,0,0


In [74]:
def get_target_column(target, df):
    return df[['headline', 'transcript','pos_sequence', 'ner_sequence','tm', target_tag]]

In [22]:
df_x = df[['headline', 'transcript','pos_sequence', 'ner_sequence','tm']]

In [76]:
df_y = df[all_tags]
# dff = list(get_target_column('culture', df))
# dff
# print(len(df_y))
df_y

,culture,politics,science,global issues,technology,design,business,biomechanics,biodiversity,media,entertainment,history,future,communication,humanity
0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0
1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0
2,1,1,1,1,0,0,1,0,0,0,0,0,0,0,0
3,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0
4,0,0,1,0,1,1,1,0,0,0,0,0,0,0,0
5,0,0,1,0,1,0,0,1,1,0,0,0,0,0,0
6,0,0,1,0,1,1,0,0,0,1,1,0,0,0,0
7,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0
8,1,0,1,1,1,1,1,0,0,1,0,0,0,0,0
9,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0


## 3.2 Perform train test split

In [79]:
from sklearn.model_selection import train_test_split
X_train, X_test, train_y, valid_y = train_test_split(df_x, df_y, random_state = 42)

In [26]:
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM, Conv1D, MaxPooling1D, concatenate
from keras.layers import GlobalMaxPooling1D
from keras.models import Model
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.layers import Input
from keras.layers.merge import Concatenate
from keras import optimizers
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

import numpy as np

Using TensorFlow backend.


## 3.3 Use word embeddings for the main transcript

In [80]:
# Extract train and test transcripts to list 
X_train_transcripts = X_train['transcript'].tolist()
X_test_transcripts = X_test['transcript'].tolist()
# Extract headline - we will use tfidf because headlines are short 
X_train_headline = X_train['headline'].tolist()
X_test_headline = X_test['headline'].tolist()
# Extract POS tags
X_train_pos_seq= X_train['pos_sequence'].tolist()
X_test_pos_seq = X_test['pos_sequence'].tolist()
# Extract NER tags
X_train_ner_seq = X_train['ner_sequence'].tolist()
X_test_ner_seq = X_test['ner_sequence'].tolist()
# Extract topic modelling arrays
X_train_tm = X_train['tm']
X_test_tm = X_test['tm']

In [81]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train_transcripts)

X_train_transcripts = tokenizer.texts_to_sequences(X_train_transcripts)
X_test_transcripts = tokenizer.texts_to_sequences(X_test_transcripts)

vocab_size = len(tokenizer.word_index) + 1

maxlen = 500 # since the average length is about there. Too long and the predicions are bad. we assume the intro has the most info

X_train_transcripts = pad_sequences(X_train_transcripts, padding='post', maxlen=maxlen)
X_test_transcripts = pad_sequences(X_test_transcripts, padding='post', maxlen=maxlen)

In [82]:
from numpy import array
from numpy import asarray
from numpy import zeros

embeddings_dictionary = dict()
glove_path = "C:/Users/JSaw/Downloads/"
glove_file = open(glove_path+'glove.6B.100d.txt', encoding="utf8")

for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary[word] = vector_dimensions
glove_file.close()

embedding_matrix = zeros((vocab_size, 100))
for word, index in tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

In [83]:
print(X_train_transcripts[1])
print(X_train_transcripts.shape)
print(type(X_train_transcripts))

[4469 4298 2877  570  399  847 1837 1132 2904 1242  663 2904 1242 3117
 2877  848   67  449  583  301 2904 1242 1447 1132  261   21  570    8
   17   68 2420 1035  169   21  382 3410  399  598  101  444  101   12
 1783  715 2211  471 1378  368 2511  715  433 4299  570  153 2843  273
  438  169   76  368  438 1423 1483 1026 1285 1035  528    6   19 2453
 2683  598 1274   78  149  598 3460  598   11  813 1721 4157   11   38
   24  153 2843  678 1437 2878 1036 2033  469    5 1514 1688   17  802
  292   25 2878 1437  814 1026 1026 2454  197  387  141  814 1326  117
 1226   14  588 1233 1462  149  241 2878    8  500  102 1548  387  141
  964   30 4300   34  241 2878   51   34 3461   51 3709  448  387  141
  544  206  241 2878  625   30   14    7 1530 1186  387  141  402   69
  241 2878   21   34   47   56  136   47  241 2878   21   77  663   43
  663 1394 4552 1437 1676   10  241 2878    7  459 2810  389  888  176
 4931 4301  626  264  285  264  285   14  567  399  241  709  389  814
 2878 

## 3.4 tfidf the headline

In [32]:
# tfidf_vect_pos = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=50)
# tfidf_vect_pos.fit(X_train_headline)

# xtrain_tfidf_headline =  tfidf_vect_pos.transform(X_train_headline)
# xtest_tfidf_headline =  tfidf_vect_pos.transform(X_test_headline)


In [33]:
# print(xtrain_tfidf_headline.shape)
# print(xtest_tfidf_headline.shape)
# print(xtrain_tfidf_headline[0])
# print(type(xtrain_tfidf_headline))

In [34]:
# tfidf_vect_pos = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
# tfidf_vect_pos.fit(df['pos_sequence'])
# tfidf_vect_ner = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
# tfidf_vect_ner.fit(df['ner_sequence'])

# xtrain_tfidf_pos =  tfidf_vect_pos.transform(X_train['pos_sequence'])
# xtest_tfidf_pos =  tfidf_vect_pos.transform(X_test['pos_sequence'])

# xtrain_tfidf_ner =  tfidf_vect_ner.transform(X_train['ner_sequence'])
# xtest_tfidf_ner =  tfidf_vect_ner.transform(X_test['ner_sequence'])

In [84]:
# Try word embeddings on the vector 
tokenizer2 = Tokenizer(num_words=100)
tokenizer2.fit_on_texts(X_train_headline)

X_train_headline = tokenizer2.texts_to_sequences(X_train_headline)
X_test_headline = tokenizer2.texts_to_sequences(X_test_headline)

vocab_size2 = len(tokenizer2.word_index) + 1

maxlen2 = 100 # since the average length is about there. Too long and the predicions are bad. we assume the intro has the most info

X_train_headline = pad_sequences(X_train_headline, padding='post', maxlen=maxlen2)
X_test_headline = pad_sequences(X_test_headline, padding='post', maxlen=maxlen2)

# 3.5 Place all tm vectors into big array

In [85]:
def compile_vectors(series,num):
    big = np.zeros((len(series),num))
    for i in range(len(series)):
        array = series.iloc[i]
        big[i] = array
        return big

In [86]:
print(type(X_train_tm))
print(type(X_test_tm))

<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


In [87]:
X_train_tm = compile_vectors(X_train_tm,15)
X_test_tm = compile_vectors(X_test_tm,15)

In [88]:
print(type(X_train_tm))
print(type(X_test_tm))
print(X_train_tm.shape)
print(X_test_tm.shape)

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(1734, 15)
(579, 15)


# 3.5 POS NER

In [89]:
print(type(X_train_pos_seq))
print(type(X_test_pos_seq))

print(type(X_train_ner_seq))
print(type(X_test_ner_seq))
print(len(X_train_pos_seq[0]),len(X_train_pos_seq[1]))

<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
5135 3703


In [90]:
ave_pos = 0
for i in range(len(X_train_pos_seq)):
    small = len(X_train_pos_seq[i])
    ave_pos += small
for j in range(len(X_test_pos_seq)):
    small = len(X_test_pos_seq[j])
    ave_pos += small
ave_pos = int(ave_pos / (len(X_train_pos_seq)+len(X_test_pos_seq)))
print(ave_pos)

4031


In [91]:
unique_pos = list(df['pos_sequence'].str.split(' ', expand=True).stack().unique())
print(unique_pos)

['VERB', 'PROPN', 'ADV', 'ADJ', 'NOUN', 'ADP', 'SCONJ', 'INTJ', 'AUX', 'PRON', 'X', 'NUM', 'DET', 'PART', 'PUNCT', 'SYM', 'CCONJ']


In [92]:
ave_ner = 0
for i in range(len(X_train_ner_seq)):
    small = len(X_train_ner_seq[i])
    ave_ner += small
for j in range(len(X_test_ner_seq)):
    small = len(X_test_ner_seq[j])
    ave_ner += small
ave_ner = int(ave_ner / (len(X_train_ner_seq)+len(X_test_ner_seq)))
print(ave_ner)

215


In [93]:
unique_ner = list(df['ner_sequence'].str.split(' ', expand=True).stack().unique())
print(unique_ner)

['PERSON', 'ORG', 'GPE', 'LOC', 'PRODUCT', 'DATE', 'EVENT', 'NORP', 'TIME', 'CARDINAL', 'MONEY', 'ORDINAL', 'PERCENT', 'FAC', 'QUANTITY', 'LANGUAGE', 'LAW', 'WORK_OF_ART', '']


In [94]:
# Try word embeddings on the vector 
tokenizer3 = Tokenizer(num_words=len(unique_pos))
tokenizer3.fit_on_texts(X_train_pos_seq)

X_train_pos_seq = tokenizer3.texts_to_sequences(X_train_pos_seq)
X_test_pos_seq = tokenizer3.texts_to_sequences(X_test_pos_seq)

vocab_size3 = len(tokenizer3.word_index) + 1

maxlen3 = ave_pos # since the average length is about there. Too long and the predicions are bad. we assume the intro has the most info

X_train_pos_seq = pad_sequences(X_train_pos_seq, padding='post', maxlen=maxlen3)
X_test_pos_seq = pad_sequences(X_test_pos_seq, padding='post', maxlen=maxlen3)

In [95]:
# Try word embeddings on the vector 
tokenizer4 = Tokenizer(num_words=len(unique_ner))
tokenizer4.fit_on_texts(X_train_ner_seq)

X_train_ner_seq = tokenizer4.texts_to_sequences(X_train_ner_seq)
X_test_ner_seq = tokenizer4.texts_to_sequences(X_test_ner_seq)

vocab_size4 = len(tokenizer4.word_index) + 1

maxlen4 = ave_ner # since the average length is about there. Too long and the predicions are bad. we assume the intro has the most info

X_train_ner_seq = pad_sequences(X_train_ner_seq, padding='post', maxlen=maxlen4)
X_test_ner_seq = pad_sequences(X_test_ner_seq, padding='post', maxlen=maxlen4)

# Model

In [96]:
from keras.utils import plot_model
# define two sets of inputs
inputA = Input(shape=(maxlen2,))
inputB = Input(shape=(maxlen,))
inputTM = Input(shape=(15,))
inputNER = Input(shape=(maxlen4,))
inputPOS = Input(shape=(maxlen3,))
 
# the first branch operates on the first input which is the headline
embedding_layer_headline = Embedding(vocab_size, 100, weights=[embedding_matrix], trainable=False)(inputA) 
#model.add(layers.Embedding(vocab_size, embedding_dim, input_length=maxlen))
#model.add(layers.Conv1D(128, 5, activation='relu'))
x = Conv1D(128, 5, activation='relu')(embedding_layer_headline)
# model.add(layers.GlobalMaxPooling1D())
x = GlobalMaxPooling1D()(x)
x = Dense(10, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(4, activation="relu")(x)
# model.add(layers.Dense(10, activation='relu'))
# model.add(layers.Dense(1, activation='sigmoid'))

# x = Dense(50, activation="relu")(inputA)
# x = Dense(4, activation="relu")(x)
x = Model(inputs=inputA, outputs=x)
 
# the second branch opreates on the second input
embedding_layer = Embedding(vocab_size, 100, weights=[embedding_matrix], trainable=False)(inputB)
y = LSTM(128)(embedding_layer)
y = Dropout(0.2)(y)
y = Dense(4, activation='relu')(y)
y = Model(inputs=inputB, outputs=y)

# third input
tm = Dense(64,activation='relu')(inputTM)
tm = Dense(4,activation='relu')(tm)
tm = Model(inputs=inputTM,outputs=tm)

# fourth input: NER
embedding_layer_ner = Embedding(vocab_size4, 20,trainable=True)(inputNER)
ner = Conv1D(128, 5, activation='relu')(embedding_layer_ner)
# model.add(layers.GlobalMaxPooling1D())
ner = GlobalMaxPooling1D()(ner)
ner = Dense(10, activation='relu')(ner)
ner = Dropout(0.2)(ner)
ner = Dense(4, activation="relu")(ner)
ner = Model(inputs=inputNER, outputs=ner)

# fifth input: POS
embedding_layer_pos = Embedding(vocab_size3, 20,trainable=True)(inputPOS)
pos = Conv1D(128, 5, activation='relu')(embedding_layer_pos)
# model.add(layers.GlobalMaxPooling1D())
pos = GlobalMaxPooling1D()(pos)
pos = Dense(10, activation='relu')(pos)
pos = Dropout(0.2)(pos)
pos = Dense(4, activation="relu")(pos)
pos = Model(inputs=inputPOS, outputs=pos)

# combine the output of the two branches
combined = concatenate([x.output, y.output, tm.output, ner.output, pos.output])
 
# apply a FC layer and then a regression prediction on the
# combined outputs
z = Dense(2, activation="relu")(combined)
z = Dense(1, activation="sigmoid")(z)
 
# our model will accept the inputs of the two branches and
# then output a single value
model = Model(inputs=[x.input, y.input, tm.input, ner.input, pos.input], outputs=z)
print(model.summary())
adam = optimizers.adam(lr=0.0001)
#model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['acc'])

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           (None, 100)          0                                            
__________________________________________________________________________________________________
input_14 (InputLayer)           (None, 215)          0                                            
__________________________________________________________________________________________________
input_15 (InputLayer)           (None, 4031)         0                                            
__________________________________________________________________________________________________
embedding_9 (Embedding)         (None, 100, 100)     4267400     input_11[0][0]                   
__________________________________________________________________________________________________
embedding_

In [97]:
print(X_train_transcripts.shape,maxlen,type(X_train_transcripts))
print(X_train_headline.shape,maxlen2,type(X_train_headline))
print(X_train_tm.shape,15,type(X_train_tm))
print(X_train_ner_seq.shape,maxlen4,type(X_train_ner_seq))
print(X_train_pos_seq.shape,maxlen3,type(X_train_pos_seq))

(1734, 500) 500 <class 'numpy.ndarray'>
(1734, 100) 100 <class 'numpy.ndarray'>
(1734, 15) 15 <class 'numpy.ndarray'>
(1734, 215) 215 <class 'numpy.ndarray'>
(1734, 4031) 4031 <class 'numpy.ndarray'>


In [98]:
plot_model(model, to_file='model_plot_cnn_tm_ner_pos.png', show_shapes=True, show_layer_names=True)

In [99]:
adam = optimizers.adam(lr=0.001)
#model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['acc'])

In [100]:
def get_tag(threshold, predictions):
    return [[1 if j > threshold else 0 for j in i.tolist()] for i in predictions]

def get_tag_flat(threshold, predictions):
    return [1 if j > threshold else 0 for i in predictions for j in i]
# predictions_flushed = get_tag(0.4)


In [101]:
def compute_tp_tn_fp_fn(y_test, y_pred, classes):
    '''
    Return:
    pre_score = {
        'tag_1': {
            'index': ,
            'tp': ,
            'tn': ,
            'fp': ,
            'fn': 
        }
    }
    '''
    # Create dictionary of tags 
    pre_score = {}
    for index_tag, tag in enumerate(classes):
        pre_score[tag] = {
            'index':index_tag,
            'tp': 0,
            'tn': 0,
            'fp': 0,
            'fn': 0
        }
    for transcript_index, transcript_value in enumerate(y_test):
        if transcript_value == y_pred[transcript_index][0] and transcript_value == 1:
            pre_score[classes[0]]['tp'] += 1
        elif transcript_value == y_pred[transcript_index][0] and transcript_value == 0:
            pre_score[classes[0]]['tn'] += 1
        elif transcript_value != y_pred[transcript_index][0] and transcript_value == 1:
            pre_score[classes[0]]['fn'] += 1
        elif transcript_value != y_pred[transcript_index][0] and transcript_value == 0:
            pre_score[classes[0]]['fp'] += 1
    return pre_score
# scores_preprocess = compute_tp_tn_fp_fn(valid_y, predictions_flushed, ['culture'])

In [118]:
def compute_precision_recall_f1(preprocessed_scores):
    for key, value in preprocessed_scores.items():
        try:
            precision = value['tp']/(value['tp']+value['fp'])
        except:
#             print('precision issue: {}'.format(key))
            precision = 0.0
        try:
            recall = value['tp']/(value['tp']+value['fn'])
        except:
#             print('recall issue: {}'.format(key))
            recall = 0.0
        try:
            f1 = (2 * precision * recall)/(precision + recall)
        except:
#             print('f1 issue: {}'.format(key))
            f1=0.0
        preprocessed_scores[key]['precision'] = round(precision,2)
        preprocessed_scores[key]['recall'] = round(recall,2)
        preprocessed_scores[key]['f1'] = round(f1,2)
    return preprocessed_scores
# final_scores = compute_precision_recall_f1(scores_preprocess)
# print(final_scores)

In [103]:
def print_full_dataframe(x):
    pd.set_option('display.max_rows', len(x))
    print(x)
    pd.reset_option('display.max_rows')

In [104]:
def format_scores_df(tag_classes, final_scores):
    precision = []
    recall = []
    f1 = []
    accuracy = []
    for index, value in enumerate(tag_classes):
        precision.append(final_scores[value]['precision'])
        recall.append(final_scores[value]['recall'])
        f1.append(final_scores[value]['f1'])
        accuracy.append((final_scores[value]['tp'] + final_scores[value]['tn'])/(final_scores[value]['tp'] + final_scores[value]['tn'] + final_scores[value]['fp'] + final_scores[value]['fn']))
    df_result = pd.DataFrame(list(zip(tag_classes, precision, recall, f1, accuracy)), 
               columns =['class', 'precision', 'recall', 'f1', 'accuracy']) 
    return df_result
# df_results = format_scores_df(['culture'], final_scores)
# print_full_dataframe(df_results)

In [142]:
def get_threshold(tag,valid_y, predictions):
    highest_f1 = 0
    f1_i = []
    highest_accuracy_f1 = 0
    accuracy_f1_i = []
    highest_accuracy = 0
    accuracy_i = []
    for i in range(0, 100):
        i = i/100
    #     print(i)
        predictions_flushed = get_tag(i,predictions)
        scores_preprocess = compute_tp_tn_fp_fn(valid_y, predictions_flushed, [tag])
        final_scores = compute_precision_recall_f1(scores_preprocess)
    #     print(final_scores)
        df_results = format_scores_df([tag], final_scores)
        print(df_results)
        f1 = final_scores[tag]['f1']
        accuracy = df_results.accuracy[0]

        if f1 > highest_f1:
            highest_f1 = f1
            f1_i = [i]
            if accuracy > highest_accuracy_f1:
                highest_accuracy_f1 = accuracy
                accuracy_f1_i = [i]
            elif accuracy == highest_accuracy_f1:
                accuracy_f1_i.append(i)
        elif f1 == highest_f1:
            f1_i.append(i)
            if accuracy > highest_accuracy_f1:
                highest_accuracy_f1 = accuracy
                accuracy_f1_i = [i]
            elif accuracy == highest_accuracy_f1:
                accuracy_f1_i.append(i)

        if accuracy > highest_accuracy:
            highest_accuracy = accuracy
            accuracy_i = [i]
        elif accuracy == highest_accuracy:
            accuracy_i.append(i)

    #     print('\n')

#     print(highest_f1,f1_i)
#     print(highest_accuracy_f1,accuracy_f1_i)
#     print(highest_accuracy,accuracy_i)
    return highest_f1,f1_i,highest_accuracy_f1,accuracy_f1_i,highest_accuracy,accuracy_i

In [106]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import roc_curve, roc_auc_score
import matplotlib.pyplot as plt
def evaluate_on_training_set(y_test, y_pred):
  # Calculate AUC
  print("AUC is: ", roc_auc_score(y_test, y_pred))
  # print out recall and precision
  print(classification_report(y_test, y_pred))
  # print out confusion matrix
  print("Confusion Matrix: \n", confusion_matrix(y_test, y_pred))
  # # calculate points for ROC curve
  fpr, tpr, thresholds = roc_curve(y_test, y_pred)
  # Plot ROC curve
  plt.plot(fpr, tpr, label='ROC curve (area = %0.3f)' % roc_auc_score(y_test, y_pred)) 
  plt.plot([0, 1], [0, 1], 'k--') # random predictions curve
  plt.xlim([0.0, 1.0])
  plt.ylim([0.0, 1.0])
  plt.xlabel('False Positive Rate or (1 - Specifity)')
  plt.ylabel('True Positive Rate or (Sensitivity)')
  plt.title('Receiver Operating Characteristic')

In [ ]:
# evaluate_on_training_set(valid_y, get_tag_flat(0.35))
# #print(valid_y)

In [139]:
def find_threshold(tag):
    train_y_tag = train_y[tag]
    valid_y_tag = valid_y[tag]
    history = model.fit([X_train_headline, X_train_transcripts, X_train_tm, X_train_ner_seq, X_train_pos_seq], train_y_tag, batch_size=32, epochs=4, verbose=1, validation_split=0.2)
    predictions = model.predict([X_test_headline, X_test_transcripts, X_test_tm, X_test_ner_seq, X_train_pos_seq])
    highest_f1,f1_i,highest_accuracy_f1,accuracy_f1_i,highest_accuracy,accuracy_i = get_threshold(tag,valid_y_tag,predictions)
    print(highest_f1,f1_i,highest_accuracy_f1,accuracy_f1_i,highest_accuracy,accuracy_i)

In [120]:
tag_results = {}
for i in range(len(all_tags)):
    tag = all_tags[i]
    print(tag)
    train_y_tag = train_y[tag]
    valid_y_tag = valid_y[tag]
    history = model.fit([X_train_headline, X_train_transcripts, X_train_tm, X_train_ner_seq, X_train_pos_seq], train_y_tag, batch_size=32, epochs=4, verbose=1, validation_split=0.2)
    predictions = model.predict([X_test_headline, X_test_transcripts, X_test_tm, X_test_ner_seq, X_train_pos_seq])
    highest_f1,f1_i,highest_accuracy_f1,accuracy_f1_i,highest_accuracy,accuracy_i = get_threshold(tag,valid_y_tag,predictions)
    print(highest_f1,f1_i,highest_accuracy_f1,accuracy_f1_i,highest_accuracy,accuracy_i)
    tag_results[tag] = [highest_f1,f1_i,highest_accuracy_f1,accuracy_f1_i,highest_accuracy,accuracy_i]

culture
Train on 1387 samples, validate on 347 samples
Epoch 1/4
1387/1387 [==============================] - ETA: 38s - loss: 0.7638 - acc: 0.37 - ETA: 36s - loss: 0.7385 - acc: 0.43 - ETA: 38s - loss: 0.7217 - acc: 0.47 - ETA: 36s - loss: 0.7039 - acc: 0.52 - ETA: 36s - loss: 0.7083 - acc: 0.51 - ETA: 35s - loss: 0.7070 - acc: 0.51 - ETA: 33s - loss: 0.7007 - acc: 0.53 - ETA: 32s - loss: 0.7086 - acc: 0.51 - ETA: 32s - loss: 0.7035 - acc: 0.52 - ETA: 31s - loss: 0.7019 - acc: 0.52 - ETA: 30s - loss: 0.7007 - acc: 0.53 - ETA: 29s - loss: 0.7028 - acc: 0.52 - ETA: 28s - loss: 0.7016 - acc: 0.52 - ETA: 26s - loss: 0.7042 - acc: 0.52 - ETA: 26s - loss: 0.7065 - acc: 0.51 - ETA: 25s - loss: 0.7077 - acc: 0.51 - ETA: 24s - loss: 0.7050 - acc: 0.52 - ETA: 23s - loss: 0.7020 - acc: 0.52 - ETA: 22s - loss: 0.7032 - acc: 0.52 - ETA: 21s - loss: 0.7037 - acc: 0.52 - ETA: 20s - loss: 0.7036 - acc: 0.52 - ETA: 19s - loss: 0.7034 - acc: 0.52 - ETA: 19s - loss: 0.7033 - acc: 0.52 - ETA: 18s - loss:

1387/1387 [==============================] - ETA: 45s - loss: 0.6019 - acc: 0.81 - ETA: 41s - loss: 0.5862 - acc: 0.85 - ETA: 42s - loss: 0.5774 - acc: 0.88 - ETA: 40s - loss: 0.5783 - acc: 0.88 - ETA: 38s - loss: 0.5788 - acc: 0.88 - ETA: 37s - loss: 0.5792 - acc: 0.88 - ETA: 36s - loss: 0.5794 - acc: 0.87 - ETA: 35s - loss: 0.5835 - acc: 0.86 - ETA: 34s - loss: 0.5831 - acc: 0.86 - ETA: 33s - loss: 0.5839 - acc: 0.86 - ETA: 32s - loss: 0.5825 - acc: 0.86 - ETA: 31s - loss: 0.5814 - acc: 0.87 - ETA: 30s - loss: 0.5788 - acc: 0.87 - ETA: 28s - loss: 0.5796 - acc: 0.87 - ETA: 28s - loss: 0.5773 - acc: 0.88 - ETA: 27s - loss: 0.5761 - acc: 0.88 - ETA: 25s - loss: 0.5755 - acc: 0.88 - ETA: 25s - loss: 0.5751 - acc: 0.88 - ETA: 23s - loss: 0.5734 - acc: 0.89 - ETA: 22s - loss: 0.5736 - acc: 0.89 - ETA: 22s - loss: 0.5717 - acc: 0.89 - ETA: 21s - loss: 0.5703 - acc: 0.90 - ETA: 19s - loss: 0.5696 - acc: 0.90 - ETA: 19s - loss: 0.5689 - acc: 0.90 - ETA: 18s - loss: 0.5682 - acc: 0.90 - ETA: 

1387/1387 [==============================] - ETA: 39s - loss: 0.7154 - acc: 0.53 - ETA: 38s - loss: 0.8136 - acc: 0.35 - ETA: 39s - loss: 0.8285 - acc: 0.33 - ETA: 39s - loss: 0.8316 - acc: 0.32 - ETA: 38s - loss: 0.8406 - acc: 0.31 - ETA: 37s - loss: 0.8317 - acc: 0.32 - ETA: 35s - loss: 0.8305 - acc: 0.33 - ETA: 34s - loss: 0.8295 - acc: 0.33 - ETA: 33s - loss: 0.8308 - acc: 0.32 - ETA: 33s - loss: 0.8318 - acc: 0.32 - ETA: 32s - loss: 0.8358 - acc: 0.32 - ETA: 30s - loss: 0.8347 - acc: 0.32 - ETA: 29s - loss: 0.8310 - acc: 0.32 - ETA: 29s - loss: 0.8278 - acc: 0.33 - ETA: 29s - loss: 0.8274 - acc: 0.33 - ETA: 28s - loss: 0.8271 - acc: 0.33 - ETA: 28s - loss: 0.8246 - acc: 0.34 - ETA: 27s - loss: 0.8235 - acc: 0.34 - ETA: 26s - loss: 0.8205 - acc: 0.34 - ETA: 25s - loss: 0.8232 - acc: 0.34 - ETA: 25s - loss: 0.8230 - acc: 0.34 - ETA: 23s - loss: 0.8253 - acc: 0.33 - ETA: 22s - loss: 0.8220 - acc: 0.34 - ETA: 21s - loss: 0.8182 - acc: 0.35 - ETA: 20s - loss: 0.8162 - acc: 0.35 - ETA: 

1387/1387 [==============================] - ETA: 50s - loss: 0.6257 - acc: 0.71 - ETA: 46s - loss: 0.6319 - acc: 0.70 - ETA: 42s - loss: 0.6051 - acc: 0.77 - ETA: 40s - loss: 0.6072 - acc: 0.76 - ETA: 38s - loss: 0.6134 - acc: 0.75 - ETA: 36s - loss: 0.6258 - acc: 0.71 - ETA: 36s - loss: 0.6206 - acc: 0.73 - ETA: 36s - loss: 0.6274 - acc: 0.71 - ETA: 34s - loss: 0.6273 - acc: 0.71 - ETA: 33s - loss: 0.6235 - acc: 0.72 - ETA: 31s - loss: 0.6237 - acc: 0.72 - ETA: 30s - loss: 0.6249 - acc: 0.72 - ETA: 30s - loss: 0.6269 - acc: 0.71 - ETA: 29s - loss: 0.6277 - acc: 0.71 - ETA: 28s - loss: 0.6268 - acc: 0.71 - ETA: 27s - loss: 0.6275 - acc: 0.71 - ETA: 25s - loss: 0.6289 - acc: 0.71 - ETA: 24s - loss: 0.6294 - acc: 0.71 - ETA: 24s - loss: 0.6286 - acc: 0.71 - ETA: 23s - loss: 0.6272 - acc: 0.71 - ETA: 22s - loss: 0.6248 - acc: 0.72 - ETA: 20s - loss: 0.6254 - acc: 0.72 - ETA: 19s - loss: 0.6276 - acc: 0.71 - ETA: 18s - loss: 0.6275 - acc: 0.71 - ETA: 18s - loss: 0.6274 - acc: 0.71 - ETA: 

1387/1387 [==============================] - ETA: 49s - loss: 0.6323 - acc: 0.68 - ETA: 42s - loss: 0.6396 - acc: 0.67 - ETA: 43s - loss: 0.6667 - acc: 0.61 - ETA: 41s - loss: 0.6618 - acc: 0.62 - ETA: 40s - loss: 0.6647 - acc: 0.61 - ETA: 38s - loss: 0.6642 - acc: 0.61 - ETA: 37s - loss: 0.6702 - acc: 0.60 - ETA: 35s - loss: 0.6691 - acc: 0.60 - ETA: 35s - loss: 0.6749 - acc: 0.59 - ETA: 34s - loss: 0.6780 - acc: 0.59 - ETA: 32s - loss: 0.6752 - acc: 0.59 - ETA: 31s - loss: 0.6704 - acc: 0.60 - ETA: 30s - loss: 0.6686 - acc: 0.61 - ETA: 29s - loss: 0.6691 - acc: 0.60 - ETA: 28s - loss: 0.6686 - acc: 0.61 - ETA: 27s - loss: 0.6710 - acc: 0.60 - ETA: 26s - loss: 0.6704 - acc: 0.60 - ETA: 25s - loss: 0.6700 - acc: 0.60 - ETA: 24s - loss: 0.6664 - acc: 0.61 - ETA: 23s - loss: 0.6654 - acc: 0.61 - ETA: 22s - loss: 0.6631 - acc: 0.62 - ETA: 21s - loss: 0.6631 - acc: 0.62 - ETA: 20s - loss: 0.6630 - acc: 0.62 - ETA: 19s - loss: 0.6623 - acc: 0.62 - ETA: 18s - loss: 0.6629 - acc: 0.62 - ETA: 

...

1387/1387 [==============================] - ETA: 49s - loss: 0.2736 - acc: 0.93 - ETA: 47s - loss: 0.3084 - acc: 0.92 - ETA: 46s - loss: 0.3158 - acc: 0.91 - ETA: 48s - loss: 0.3562 - acc: 0.89 - ETA: 46s - loss: 0.3255 - acc: 0.91 - ETA: 45s - loss: 0.3256 - acc: 0.91 - ETA: 43s - loss: 0.3157 - acc: 0.91 - ETA: 42s - loss: 0.3103 - acc: 0.91 - ETA: 42s - loss: 0.3436 - acc: 0.90 - ETA: 40s - loss: 0.3276 - acc: 0.91 - ETA: 39s - loss: 0.3211 - acc: 0.91 - ETA: 37s - loss: 0.3394 - acc: 0.90 - ETA: 37s - loss: 0.3517 - acc: 0.90 - ETA: 36s - loss: 0.3607 - acc: 0.89 - ETA: 34s - loss: 0.3676 - acc: 0.89 - ETA: 33s - loss: 0.3653 - acc: 0.89 - ETA: 32s - loss: 0.3634 - acc: 0.89 - ETA: 31s - loss: 0.3698 - acc: 0.89 - ETA: 29s - loss: 0.3723 - acc: 0.89 - ETA: 28s - loss: 0.3727 - acc: 0.89 - ETA: 27s - loss: 0.3776 - acc: 0.88 - ETA: 26s - loss: 0.3836 - acc: 0.88 - ETA: 25s - loss: 0.3848 - acc: 0.88 - ETA: 23s - loss: 0.3848 - acc: 0.88 - ETA: 22s - loss: 0.3847 - acc: 0.88 - ETA: 

1387/1387 [==============================] - ETA: 48s - loss: 0.3894 - acc: 0.84 - ETA: 47s - loss: 0.5876 - acc: 0.76 - ETA: 52s - loss: 0.5505 - acc: 0.78 - ETA: 49s - loss: 0.5122 - acc: 0.80 - ETA: 47s - loss: 0.4836 - acc: 0.82 - ETA: 45s - loss: 0.4674 - acc: 0.83 - ETA: 43s - loss: 0.5008 - acc: 0.82 - ETA: 44s - loss: 0.5483 - acc: 0.80 - ETA: 42s - loss: 0.5223 - acc: 0.81 - ETA: 41s - loss: 0.5054 - acc: 0.81 - ETA: 39s - loss: 0.4943 - acc: 0.82 - ETA: 39s - loss: 0.4999 - acc: 0.82 - ETA: 37s - loss: 0.5032 - acc: 0.81 - ETA: 36s - loss: 0.4932 - acc: 0.82 - ETA: 35s - loss: 0.4912 - acc: 0.82 - ETA: 33s - loss: 0.4855 - acc: 0.82 - ETA: 32s - loss: 0.4788 - acc: 0.83 - ETA: 31s - loss: 0.4763 - acc: 0.83 - ETA: 30s - loss: 0.4729 - acc: 0.83 - ETA: 28s - loss: 0.4677 - acc: 0.83 - ETA: 27s - loss: 0.4671 - acc: 0.83 - ETA: 26s - loss: 0.4702 - acc: 0.83 - ETA: 25s - loss: 0.4669 - acc: 0.83 - ETA: 23s - loss: 0.4612 - acc: 0.84 - ETA: 22s - loss: 0.4639 - acc: 0.83 - ETA: 

1387/1387 [==============================] - ETA: 55s - loss: 0.2465 - acc: 0.96 - ETA: 50s - loss: 0.2681 - acc: 0.95 - ETA: 48s - loss: 0.3143 - acc: 0.92 - ETA: 50s - loss: 0.3384 - acc: 0.90 - ETA: 48s - loss: 0.3293 - acc: 0.91 - ETA: 46s - loss: 0.3373 - acc: 0.90 - ETA: 44s - loss: 0.3435 - acc: 0.90 - ETA: 46s - loss: 0.3369 - acc: 0.90 - ETA: 45s - loss: 0.3303 - acc: 0.90 - ETA: 44s - loss: 0.3339 - acc: 0.90 - ETA: 42s - loss: 0.3420 - acc: 0.90 - ETA: 41s - loss: 0.3338 - acc: 0.90 - ETA: 40s - loss: 0.3255 - acc: 0.91 - ETA: 38s - loss: 0.3260 - acc: 0.91 - ETA: 36s - loss: 0.3321 - acc: 0.90 - ETA: 35s - loss: 0.3351 - acc: 0.90 - ETA: 34s - loss: 0.3330 - acc: 0.90 - ETA: 32s - loss: 0.3305 - acc: 0.90 - ETA: 31s - loss: 0.3297 - acc: 0.90 - ETA: 29s - loss: 0.3315 - acc: 0.90 - ETA: 28s - loss: 0.3277 - acc: 0.90 - ETA: 27s - loss: 0.3247 - acc: 0.90 - ETA: 26s - loss: 0.3295 - acc: 0.90 - ETA: 24s - loss: 0.3333 - acc: 0.90 - ETA: 23s - loss: 0.3339 - acc: 0.90 - ETA: 

1387/1387 [==============================] - ETA: 38s - loss: 0.2654 - acc: 0.93 - ETA: 37s - loss: 0.2213 - acc: 0.95 - ETA: 36s - loss: 0.2566 - acc: 0.93 - ETA: 35s - loss: 0.2498 - acc: 0.93 - ETA: 34s - loss: 0.2777 - acc: 0.92 - ETA: 33s - loss: 0.2591 - acc: 0.93 - ETA: 32s - loss: 0.2460 - acc: 0.93 - ETA: 31s - loss: 0.2616 - acc: 0.92 - ETA: 30s - loss: 0.2600 - acc: 0.93 - ETA: 30s - loss: 0.2577 - acc: 0.93 - ETA: 29s - loss: 0.2618 - acc: 0.92 - ETA: 28s - loss: 0.2609 - acc: 0.92 - ETA: 27s - loss: 0.2654 - acc: 0.92 - ETA: 26s - loss: 0.2857 - acc: 0.91 - ETA: 25s - loss: 0.2903 - acc: 0.91 - ETA: 25s - loss: 0.2786 - acc: 0.92 - ETA: 24s - loss: 0.2860 - acc: 0.91 - ETA: 23s - loss: 0.2829 - acc: 0.92 - ETA: 22s - loss: 0.2778 - acc: 0.92 - ETA: 21s - loss: 0.2758 - acc: 0.92 - ETA: 20s - loss: 0.2730 - acc: 0.92 - ETA: 19s - loss: 0.2760 - acc: 0.92 - ETA: 18s - loss: 0.2714 - acc: 0.92 - ETA: 17s - loss: 0.2740 - acc: 0.92 - ETA: 16s - loss: 0.2729 - acc: 0.92 - ETA: 

1387/1387 [==============================] - ETA: 49s - loss: 0.2619 - acc: 0.93 - ETA: 48s - loss: 0.1997 - acc: 0.95 - ETA: 51s - loss: 0.2010 - acc: 0.94 - ETA: 50s - loss: 0.2582 - acc: 0.92 - ETA: 48s - loss: 0.3078 - acc: 0.90 - ETA: 46s - loss: 0.2900 - acc: 0.91 - ETA: 44s - loss: 0.3028 - acc: 0.90 - ETA: 45s - loss: 0.3378 - acc: 0.89 - ETA: 43s - loss: 0.3695 - acc: 0.88 - ETA: 41s - loss: 0.3538 - acc: 0.89 - ETA: 40s - loss: 0.3519 - acc: 0.89 - ETA: 39s - loss: 0.3430 - acc: 0.89 - ETA: 38s - loss: 0.3335 - acc: 0.89 - ETA: 37s - loss: 0.3348 - acc: 0.89 - ETA: 35s - loss: 0.3440 - acc: 0.89 - ETA: 34s - loss: 0.3282 - acc: 0.90 - ETA: 33s - loss: 0.3225 - acc: 0.90 - ETA: 32s - loss: 0.3177 - acc: 0.90 - ETA: 30s - loss: 0.3321 - acc: 0.89 - ETA: 29s - loss: 0.3205 - acc: 0.90 - ETA: 28s - loss: 0.3152 - acc: 0.90 - ETA: 27s - loss: 0.3081 - acc: 0.90 - ETA: 25s - loss: 0.3089 - acc: 0.90 - ETA: 24s - loss: 0.3137 - acc: 0.90 - ETA: 22s - loss: 0.3077 - acc: 0.90 - ETA: 

In [121]:
results = pd.DataFrame.from_dict(tag_results)
results

,culture,politics,science,global issues,technology,design,business,biomechanics,biodiversity,media,entertainment,history,future,communication,humanity
0,0.68,0.11,0.8,0.47,0.49,0.34,0.32,0.24,0.2,0.22,0.3,0.31,0.2,0.18,0.22
1,"[0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07...","[0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07...","[0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07...","[0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07...","[0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07...","[0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07...",[0.16],[0.12],[0.04],"[0.05, 0.08]","[0.16, 0.18]","[0.11, 0.12, 0.13, 0.14, 0.15, 0.18]","[0.09, 0.1]","[0.06, 0.07]","[0.0, 0.01, 0.02, 0.03, 0.04, 0.07]"
2,0.518135,0.0604491,0.663212,0.303972,0.324698,0.207254,0.601036,0.592401,0.442142,0.214162,0.75475,0.426598,0.459413,0.488774,0.457686
3,"[0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07...","[0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07...","[0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07...","[0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07...","[0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07...","[0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07...",[0.16],[0.12],[0.04],[0.08],[0.18],[0.18],[0.1],[0.07],[0.07]
4,0.518135,0.939551,0.663212,0.696028,0.675302,0.792746,0.841105,0.901554,0.903282,0.882556,0.861831,0.834197,0.906736,0.908463,0.875648
5,"[0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07...","[0.37, 0.38, 0.39, 0.4, 0.41, 0.42, 0.43, 0.44...","[0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07...","[0.39, 0.4, 0.41, 0.42, 0.43, 0.44, 0.45, 0.46...","[0.38, 0.39, 0.4, 0.41, 0.42, 0.43, 0.44, 0.45...","[0.35, 0.36, 0.37, 0.38, 0.39, 0.4, 0.41, 0.42...","[0.21, 0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.2...","[0.18, 0.19, 0.2, 0.21, 0.22, 0.23, 0.24, 0.25...","[0.17, 0.18, 0.19, 0.2, 0.21, 0.22, 0.23, 0.24...","[0.18, 0.19, 0.2, 0.21, 0.22, 0.23, 0.24, 0.25...","[0.25, 0.26, 0.27, 0.28, 0.29, 0.3, 0.31, 0.32...","[0.32, 0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.3...","[0.18, 0.19, 0.2, 0.21, 0.22, 0.23, 0.24, 0.25...","[0.17, 0.18, 0.19, 0.2, 0.21, 0.22, 0.23, 0.24...","[0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2..."


In [122]:
results = results.rename(index = {0:'f1',1:'f1_index',2:'accuracy_f1',3:'accuracy_f1_index',4:'accuracy',5:'accuracy_index'})
results

,culture,politics,science,global issues,technology,design,business,biomechanics,biodiversity,media,entertainment,history,future,communication,humanity
f1,0.68,0.11,0.8,0.47,0.49,0.34,0.32,0.24,0.2,0.22,0.3,0.31,0.2,0.18,0.22
f1_index,"[0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07...","[0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07...","[0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07...","[0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07...","[0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07...","[0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07...",[0.16],[0.12],[0.04],"[0.05, 0.08]","[0.16, 0.18]","[0.11, 0.12, 0.13, 0.14, 0.15, 0.18]","[0.09, 0.1]","[0.06, 0.07]","[0.0, 0.01, 0.02, 0.03, 0.04, 0.07]"
accuracy_f1,0.518135,0.0604491,0.663212,0.303972,0.324698,0.207254,0.601036,0.592401,0.442142,0.214162,0.75475,0.426598,0.459413,0.488774,0.457686
accuracy_f1_index,"[0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07...","[0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07...","[0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07...","[0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07...","[0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07...","[0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07...",[0.16],[0.12],[0.04],[0.08],[0.18],[0.18],[0.1],[0.07],[0.07]
accuracy,0.518135,0.939551,0.663212,0.696028,0.675302,0.792746,0.841105,0.901554,0.903282,0.882556,0.861831,0.834197,0.906736,0.908463,0.875648
accuracy_index,"[0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07...","[0.37, 0.38, 0.39, 0.4, 0.41, 0.42, 0.43, 0.44...","[0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07...","[0.39, 0.4, 0.41, 0.42, 0.43, 0.44, 0.45, 0.46...","[0.38, 0.39, 0.4, 0.41, 0.42, 0.43, 0.44, 0.45...","[0.35, 0.36, 0.37, 0.38, 0.39, 0.4, 0.41, 0.42...","[0.21, 0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.2...","[0.18, 0.19, 0.2, 0.21, 0.22, 0.23, 0.24, 0.25...","[0.17, 0.18, 0.19, 0.2, 0.21, 0.22, 0.23, 0.24...","[0.18, 0.19, 0.2, 0.21, 0.22, 0.23, 0.24, 0.25...","[0.25, 0.26, 0.27, 0.28, 0.29, 0.3, 0.31, 0.32...","[0.32, 0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.3...","[0.18, 0.19, 0.2, 0.21, 0.22, 0.23, 0.24, 0.25...","[0.17, 0.18, 0.19, 0.2, 0.21, 0.22, 0.23, 0.24...","[0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2..."


In [127]:
results.dtypes

culture          object
politics         object
science          object
global issues    object
technology       object
design           object
business         object
biomechanics     object
biodiversity     object
media            object
entertainment    object
history          object
future           object
communication    object
humanity         object
dtype: object

In [129]:
results.to_csv(DATA_DIR+'multi_headline_transcript_tm_ner_pos_results.csv')

In [131]:
results_csv = pd.read_csv(DATA_DIR+'multi_headline_transcript_tm_ner_pos_results.csv',index_col = 0)
results_csv

,culture,politics,science,global issues,technology,design,business,biomechanics,biodiversity,media,entertainment,history,future,communication,humanity
f1,0.68,0.11,0.8,0.47,0.49,0.34,0.32,0.24,0.2,0.22,0.3,0.31,0.2,0.18,0.22
f1_index,"[0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07...","[0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07...","[0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07...","[0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07...","[0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07...","[0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07...",[0.16],[0.12],[0.04],"[0.05, 0.08]","[0.16, 0.18]","[0.11, 0.12, 0.13, 0.14, 0.15, 0.18]","[0.09, 0.1]","[0.06, 0.07]","[0.0, 0.01, 0.02, 0.03, 0.04, 0.07]"
accuracy_f1,0.5181347150259067,0.06044905008635579,0.6632124352331606,0.30397236614853196,0.32469775474956825,0.20725388601036268,0.6010362694300518,0.5924006908462867,0.4421416234887737,0.2141623488773748,0.7547495682210709,0.4265975820379965,0.459412780656304,0.48877374784110533,0.45768566493955093
accuracy_f1_index,"[0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07...","[0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07...","[0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07...","[0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07...","[0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07...","[0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07...",[0.16],[0.12],[0.04],[0.08],[0.18],[0.18],[0.1],[0.07],[0.07]
accuracy,0.5181347150259067,0.9395509499136442,0.6632124352331606,0.696027633851468,0.6753022452504318,0.7927461139896373,0.8411053540587219,0.9015544041450777,0.9032815198618307,0.8825561312607945,0.8618307426597582,0.8341968911917098,0.9067357512953368,0.9084628670120898,0.8756476683937824
accuracy_index,"[0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07...","[0.37, 0.38, 0.39, 0.4, 0.41, 0.42, 0.43, 0.44...","[0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07...","[0.39, 0.4, 0.41, 0.42, 0.43, 0.44, 0.45, 0.46...","[0.38, 0.39, 0.4, 0.41, 0.42, 0.43, 0.44, 0.45...","[0.35, 0.36, 0.37, 0.38, 0.39, 0.4, 0.41, 0.42...","[0.21, 0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.2...","[0.18, 0.19, 0.2, 0.21, 0.22, 0.23, 0.24, 0.25...","[0.17, 0.18, 0.19, 0.2, 0.21, 0.22, 0.23, 0.24...","[0.18, 0.19, 0.2, 0.21, 0.22, 0.23, 0.24, 0.25...","[0.25, 0.26, 0.27, 0.28, 0.29, 0.3, 0.31, 0.32...","[0.32, 0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.3...","[0.18, 0.19, 0.2, 0.21, 0.22, 0.23, 0.24, 0.25...","[0.17, 0.18, 0.19, 0.2, 0.21, 0.22, 0.23, 0.24...","[0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2..."


In [135]:
print_full_dataframe(squashed_tag_counts)


    squash15_tags  counts  no_count     ratio  overall_ratio
0         science    1467       846  1.734043       0.634241
1         culture    1155      1158  0.997409       0.499351
2      technology     787      1526  0.515727       0.340251
3   global issues     679      1634  0.415545       0.293558
4          design     477      1836  0.259804       0.206226
5         history     385      1928  0.199689       0.166450
6        business     349      1964  0.177699       0.150886
7   entertainment     285      2028  0.140533       0.123217
8           media     279      2034  0.137168       0.120623
9    biomechanics     220      2093  0.105112       0.095115
10         future     218      2095  0.104057       0.094250
11   biodiversity     218      2095  0.104057       0.094250
12       humanity     217      2096  0.103531       0.093818
13       politics     199      2114  0.094134       0.086035
14  communication     185      2128  0.086936       0.079983


In [138]:
print(results['culture']['f1_index'])
print(results['culture']['accuracy_f1_index'])

[0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2, 0.21, 0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3, 0.31, 0.32, 0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4, 0.41]
[0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2, 0.21, 0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3, 0.31, 0.32, 0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4, 0.41]


In [143]:
find_threshold('culture')

Train on 1387 samples, validate on 347 samples
Epoch 1/4
1387/1387 [==============================] - ETA: 48s - loss: 0.7071 - acc: 0.68 - ETA: 54s - loss: 0.8374 - acc: 0.60 - ETA: 50s - loss: 0.9577 - acc: 0.54 - ETA: 47s - loss: 1.0075 - acc: 0.51 - ETA: 45s - loss: 1.0240 - acc: 0.50 - ETA: 45s - loss: 1.0560 - acc: 0.47 - ETA: 44s - loss: 1.0740 - acc: 0.45 - ETA: 42s - loss: 1.0404 - acc: 0.47 - ETA: 40s - loss: 1.0575 - acc: 0.45 - ETA: 39s - loss: 1.0349 - acc: 0.46 - ETA: 38s - loss: 1.0154 - acc: 0.47 - ETA: 37s - loss: 1.0129 - acc: 0.47 - ETA: 36s - loss: 0.9947 - acc: 0.48 - ETA: 35s - loss: 0.9877 - acc: 0.48 - ETA: 33s - loss: 0.9729 - acc: 0.48 - ETA: 33s - loss: 0.9628 - acc: 0.49 - ETA: 31s - loss: 0.9587 - acc: 0.49 - ETA: 30s - loss: 0.9536 - acc: 0.49 - ETA: 29s - loss: 0.9473 - acc: 0.49 - ETA: 28s - loss: 0.9353 - acc: 0.50 - ETA: 27s - loss: 0.9264 - acc: 0.50 - ETA: 25s - loss: 0.9198 - acc: 0.50 - ETA: 24s - loss: 0.9210 - acc: 0.50 - ETA: 23s - loss: 0.9263 

0  culture       0.52     1.0  0.68  0.518135
     class  precision  recall    f1  accuracy
0  culture       0.52     1.0  0.68  0.518135
     class  precision  recall    f1  accuracy
0  culture       0.52     1.0  0.68  0.518135
     class  precision  recall    f1  accuracy
0  culture       0.52     1.0  0.68  0.518135
     class  precision  recall    f1  accuracy
0  culture       0.52     1.0  0.68  0.518135
     class  precision  recall    f1  accuracy
0  culture       0.52     1.0  0.68  0.518135
     class  precision  recall    f1  accuracy
0  culture       0.52     1.0  0.68  0.518135
     class  precision  recall    f1  accuracy
0  culture       0.52     1.0  0.68  0.518135
     class  precision  recall    f1  accuracy
0  culture       0.52     1.0  0.68  0.518135
     class  precision  recall    f1  accuracy
0  culture       0.52     1.0  0.68  0.518135
     class  precision  recall    f1  accuracy
0  culture       0.52     1.0  0.68  0.518135
     class  precision  recall    f

0.68 [0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2, 0.21, 0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3, 0.31, 0.32, 0.33, 0.34, 0.35, 0.36, 0.37] 0.5181347150259067 [0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2, 0.21, 0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3, 0.31, 0.32, 0.33, 0.34, 0.35, 0.36, 0.37] 0.5181347150259067 [0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2, 0.21, 0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3, 0.31, 0.32, 0.33, 0.34, 0.35, 0.36, 0.37]
